<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/WOOK/CodeBERT_1_N(Last_train).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cwe file/MSR+julite+Div_final.csv')

In [ ]:
df.loc[df['vul'] == 0, 'CWE ID'] = 'safe'

In [ ]:
df_cut = df['CWE ID'].value_counts()
df_cut

CWE ID
CWE-119    2800
safe       2700
CWE-20     1978
CWE-125    1681
CWE-787    1536
CWE-415    1380
CWE-399    1059
CWE-416    1057
CWE-476     912
CWE-190     735
Name: count, dtype: int64

In [ ]:
cwe_to_label = {
    'safe': 0,
    'CWE-119': 1,
    'CWE-20': 2,
    'CWE-125': 3,
    'CWE-787': 4,
    'CWE-415': 5,
    'CWE-399': 6,
    'CWE-416': 7,
    'CWE-476': 8,
    'CWE-190': 9
}
df['labels'] = df['CWE ID'].map(cwe_to_label)


In [ ]:
df

,CWE ID,vul,lang,code,labels
0,safe,0,c,void func(struct net_device *d) { isdn_net_loc...,0
1,safe,0,c,void func(isdn_net_local *lp) { struct sk_buff...,0
2,safe,0,c,static __inline__ void func(isdn_net_local *lp...,0
3,safe,0,c,"void func(char *name, struct net_device *maste...",0
4,safe,0,c,void func(char *name) { u_long flags; isdn_net...,0
...,...,...,...,...,...
15833,safe,0,c,\nvoid func(char * data);\n\nstatic void func(...,0
15834,safe,0,cpp,\nstatic void func()\n{\n long * data;\n ...,0
15835,safe,0,cpp,\nstatic void func()\n{\n int64_t * data;\n...,0
15836,safe,0,c,static const void func(OnigEncoding mbctype) {...,0


In [ ]:
import pandas as pd
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from transformers import RobertaForSequenceClassification, RobertaTokenizer, TrainingArguments, Trainer

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base-mlm", num_labels = 11)
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base-mlm")
batch_size = 16
epoch_num = 1000
MAX_LEN = 512
learning_rate = 2e-5

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base-mlm and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
import re

def preprocess(df, file_name):
    # 멀티 라인 주석 제거
    df['code'] = df['code'].replace(re.compile(r'/\*.*?\*/', re.DOTALL), "", regex=True)
    # 싱글 라인 주석 제거
    df['code'] = df['code'].replace(re.compile(r'//.*?\n'), "", regex=True)
    # angle brackets를 사용하는 include 제거
    df['code'] = df['code'].replace(re.compile(r'#include .*?\n'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#include "*.*?\n'), "", regex=True)
    # 매크로 정의 제거
    df['code'] = df['code'].replace(re.compile(r'#define .*?\n'), "", regex=True)
    # 전처리 지시문 제거
    df['code'] = df['code'].replace(re.compile(r'#undef\s+\w+'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#if\s+\w+'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#else\s+\w+'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#elif\s+\w+'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#endif'), "", regex=True)
    # 탭과 여러 공백을 하나의 공백으로
    df['code'] = df['code'].replace(re.compile(r'[\t ]+'), " ", regex=True)
    # 여러 줄바꿈을 하나로
    df['code'] = df['code'].replace(re.compile(r'\n\s*\n'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'\n'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'return*.*?;'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'return;'), "", regex=True)
    # void func 로 변경
    df['code'] = df['code'].replace(re.compile(r'\b([a-zA-Z_]\w*)\s+([a-zA-Z_]\w*)\s*\(([^)]*)\)\s*{'), r"void func(\3){", regex=True)
    # 데이터프레임을 CSV 파일로 저장
    df.to_csv(file_name, index=False)
    return df  # 수정된 데이터프레임 반환

def tokenized(examples):
  return tokenizer(examples['code'], pad_to_max_length=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [ ]:
preprocess(df,"preprocess.csv")
dataset = load_dataset("csv",data_files="preprocess.csv")['train']
encoded_dataset = dataset.map(tokenized,remove_columns=['code'],batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/15838 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
encoded_dataset = encoded_dataset.train_test_split(0.3,seed=100)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.99), eps=1e-8,
            weight_decay=0.1)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # 각 클래스에 대한 F1 스코어 계산
    f1_per_class = f1_score(labels, predictions, average=None, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ])

    # 가중 평균, 매크로 평균, 마이크로 평균 F1 스코어 계산
    f1_weighted = f1_score(labels, predictions, average='weighted')
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_micro = f1_score(labels, predictions, average='micro')
    recall_weighted = recall_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # 결과를 딕셔너리 형태로 저장
    metrics = {
        'accuracy': accuracy,
        'recall_weighted': recall_weighted,
        'f1_weighted': f1_weighted,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
    }

    # 클래스별 F1 스코어를 딕셔너리에 추가
    for i, f1 in enumerate(f1_per_class):
        metrics[f'f1_class_{i}'] = f1

    return metrics

In [ ]:
from transformers import TrainerCallback
class CustomTrainerCallback(TrainerCallback):
    def __init__(self, trainer, eval_dataset, early_stopping_patience=5):
        self.trainer = trainer
        self.eval_dataset = eval_dataset
        self.early_stopping_patience = early_stopping_patience
        self.best_loss = float('inf')
        self.patience_counter = 0
        self.incorrect_samples = []

    def on_evaluate(self, args, state, control, **kwargs):
        eval_loss = kwargs.get('metrics', {}).get('eval_loss')

        if eval_loss is not None:
            if eval_loss < self.best_loss:
                self.best_loss = eval_loss
                self.patience_counter = 0
            else:
                self.patience_counter += 1

            if self.patience_counter >= self.early_stopping_patience:
                print(f"No improvement in {self.early_stopping_patience} evaluation steps. Stopping training early.")
                control.should_training_stop = True

        # Evaluate를 위한 DataLoader 설정 및 평가 로직
        eval_dataloader = self.trainer.get_eval_dataloader(self.eval_dataset)
        self.trainer.model.eval()
        for batch in eval_dataloader:
            batch = {k: v.to(self.trainer.args.device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = self.trainer.model(**batch)
                predictions = torch.argmax(outputs.logits, dim=-1)
                labels = batch['labels']
                incorrect_mask = predictions != labels

            if incorrect_mask.any():
                incorrect_samples = {k: v[incorrect_mask].cpu().numpy() for k, v in batch.items()}
                self.incorrect_samples.append(incorrect_samples)

        print("Collected incorrect predictions for further analysis.")

In [ ]:
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import KFold

# 데이터셋 준비 및 트레이너 설정
full_dataset = load_dataset("csv", data_files="preprocess.csv")['train']
full_dataset = full_dataset.map(tokenized, batched=True, remove_columns=['code'])
kf = KFold(n_splits=7, shuffle=True, random_state=42)

fold_results = []
for fold, (train_idx, test_idx) in enumerate(kf.split(full_dataset)):
    print(f"Starting fold {fold + 1}/{kf.n_splits}")  # 현재 폴드 번호 출력

    train_dataset = full_dataset.select(train_idx)
    test_dataset = full_dataset.select(test_idx)

    model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base-mlm", num_labels=11)
    training_args = TrainingArguments(
        output_dir=f'./results_fold_{fold}',
        num_train_epochs=7,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        evaluation_strategy="steps",
        eval_steps=500,
        logging_dir=f'./logs_fold_{fold}',
        load_best_model_at_end=True,
        metric_for_best_model='loss'
    )

    # 트레이너 초기화
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # 콜백 정의 및 추가
    callback = CustomTrainerCallback(trainer, test_dataset, early_stopping_patience=5)
    trainer.add_callback(callback)

    # 트레이너 학습
    trainer.train()
    trainer.save_model(f'/content/drive/MyDrive/codebert_fold_{fold}')
    print(f"{fold + 1}/{kf.n_splits} 폴드 완료")

Map:   0%|          | 0/15838 [00:00<?, ? examples/s]

Starting fold 1/7


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base-mlm and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,Recall Weighted,F1 Weighted,F1 Macro,F1 Micro,F1 Class 0,F1 Class 1,F1 Class 2,F1 Class 3,F1 Class 4,F1 Class 5,F1 Class 6,F1 Class 7,F1 Class 8,F1 Class 9
500,1.915100,1.534189,0.481662,0.481662,0.450388,0.401917,0.481662,0.657005,0.458034,0.386667,0.418269,0.439201,0.890080,0.192661,0.201005,0.078947,0.297297
1000,1.330100,1.205469,0.600530,0.600530,0.596466,0.573268,0.600530,0.781609,0.529492,0.501672,0.576271,0.592992,0.929648,0.443548,0.524823,0.334405,0.518219
1500,1.029000,1.061996,0.669907,0.669907,0.665518,0.643667,0.669907,0.842912,0.632171,0.609901,0.620000,0.579137,0.927318,0.581940,0.591241,0.379487,0.672566
2000,0.767800,1.009644,0.697746,0.697746,0.699729,0.684510,0.697746,0.854545,0.660025,0.606241,0.685106,0.634761,0.936061,0.615942,0.639344,0.520000,0.693069
2500,0.619600,0.984042,0.720283,0.720283,0.718186,0.710262,0.720283,0.839657,0.652632,0.705455,0.674419,0.637363,0.934673,0.631944,0.688889,0.614815,0.722772
3000,0.420000,1.028676,0.739284,0.739284,0.738262,0.729502,0.739284,0.861076,0.679565,0.701826,0.703934,0.663573,0.944162,0.707792,0.687500,0.618321,0.727273
3500,0.348300,1.126470,0.741935,0.741935,0.740431,0.734947,0.741935,0.837037,0.689737,0.713514,0.696970,0.674260,0.939394,0.702341,0.683128,0.648649,0.764444
4000,0.243100,1.182069,0.748564,0.748564,0.749497,0.738819,0.748564,0.867374,0.712294,0.694491,0.723849,0.696296,0.944162,0.670769,0.668919,0.654275,0.755760
4500,0.210700,1.245124,0.746354,0.746354,0.746304,0.738858,0.746354,0.858958,0.707224,0.675119,0.725664,0.669903,0.949239,0.673333,0.708955,0.680851,0.739336
5000,0.170600,1.226119,0.763588,0.763588,0.763312,0.755652,0.763588,0.874189,0.722359,0.736462,0.711297,0.694253,0.936709,0.681672,0.721805,0.715867,0.761905


Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
No improvement in 5 evaluation steps. Stopping training early.
Collected incorrect predictions for further analysis.
1/7 폴드 완료
Starting fold 2/7


...

Step,Training Loss,Validation Loss,Accuracy,Recall Weighted,F1 Weighted,F1 Macro,F1 Micro,F1 Class 0,F1 Class 1,F1 Class 2,F1 Class 3,F1 Class 4,F1 Class 5,F1 Class 6,F1 Class 7,F1 Class 8,F1 Class 9
500,1.945800,1.534530,0.475243,0.475243,0.431496,0.364211,0.475243,0.656379,0.457953,0.328160,0.406926,0.442049,0.934625,0.232932,0.050955,0.015625,0.116505
1000,1.358300,1.197308,0.625995,0.625995,0.616862,0.580745,0.625995,0.813842,0.583893,0.517172,0.536050,0.650746,0.957547,0.516517,0.441065,0.331361,0.459259
1500,1.044900,1.025622,0.675066,0.675066,0.674572,0.644873,0.675066,0.846658,0.626415,0.628205,0.602620,0.632850,0.950000,0.635135,0.558140,0.450185,0.518519
2000,0.757600,1.054630,0.695402,0.695402,0.693990,0.666683,0.695402,0.843403,0.641148,0.659048,0.658436,0.659155,0.938776,0.671010,0.531792,0.538813,0.525253
2500,0.626600,1.021850,0.716622,0.716622,0.717521,0.697623,0.716622,0.855615,0.635044,0.700730,0.678652,0.666667,0.956120,0.694118,0.625000,0.580952,0.583333
3000,0.419200,1.026855,0.720159,0.720159,0.719661,0.702113,0.720159,0.852055,0.659841,0.706897,0.677228,0.650919,0.921739,0.671053,0.639706,0.591093,0.650602
3500,0.351100,1.064670,0.742263,0.742263,0.742382,0.723242,0.742263,0.867568,0.688327,0.713043,0.731707,0.666667,0.948081,0.716049,0.623729,0.646809,0.630435
4000,0.253100,1.126445,0.744474,0.744474,0.741658,0.726284,0.744474,0.844893,0.693529,0.689189,0.724806,0.679245,0.961625,0.746269,0.635659,0.651786,0.635838
4500,0.189700,1.252285,0.742263,0.742263,0.741074,0.725252,0.742263,0.864295,0.679245,0.717608,0.714859,0.666667,0.959091,0.688047,0.637363,0.666667,0.658683
5000,0.178000,1.221611,0.755526,0.755526,0.753391,0.737099,0.755526,0.851995,0.709838,0.735593,0.722008,0.677003,0.961451,0.735632,0.648649,0.692982,0.635838


Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
No improvement in 5 evaluation steps. Stopping training early.
Collected incorrect predictions for further analysis.
6/7 폴드 완료
Starting fold 7/7


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base-mlm and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,Recall Weighted,F1 Weighted,F1 Macro,F1 Micro,F1 Class 0,F1 Class 1,F1 Class 2,F1 Class 3,F1 Class 4,F1 Class 5,F1 Class 6,F1 Class 7,F1 Class 8,F1 Class 9
500,1.934200,1.547569,0.480990,0.480990,0.443355,0.389414,0.480990,0.630336,0.459794,0.410619,0.381339,0.463415,0.869806,0.259067,0.191011,0.059259,0.169492
1000,1.352500,1.263640,0.598585,0.598585,0.591928,0.564448,0.598585,0.790464,0.561049,0.497059,0.553991,0.550588,0.911528,0.504587,0.443478,0.271739,0.560000
1500,1.039700,1.070630,0.662246,0.662246,0.658478,0.641024,0.662246,0.820639,0.607676,0.596000,0.585153,0.627002,0.925532,0.643110,0.574468,0.405660,0.625000
2000,0.778500,1.091832,0.675950,0.675950,0.676640,0.659054,0.675950,0.832041,0.631702,0.605769,0.593074,0.660714,0.940860,0.651982,0.609589,0.462185,0.602620
2500,0.630100,1.014715,0.697171,0.697171,0.696509,0.682977,0.697171,0.834184,0.645933,0.639316,0.647948,0.666667,0.935829,0.669231,0.604317,0.516667,0.669683
3000,0.424000,1.093994,0.712644,0.712644,0.712061,0.696111,0.712644,0.832061,0.670688,0.681967,0.651885,0.690307,0.924675,0.730627,0.635036,0.536170,0.607692
3500,0.350600,1.120465,0.728559,0.728559,0.730092,0.711825,0.728559,0.859459,0.711514,0.672986,0.691304,0.686316,0.936508,0.704225,0.673077,0.540323,0.642534
4000,0.255300,1.149778,0.729443,0.729443,0.730101,0.715793,0.729443,0.835897,0.695652,0.691928,0.685225,0.704883,0.931937,0.757576,0.661922,0.578125,0.614786
4500,0.192200,1.281803,0.737401,0.737401,0.737796,0.720826,0.737401,0.853816,0.718987,0.702875,0.688742,0.687898,0.931579,0.775194,0.666667,0.549020,0.633484
5000,0.180400,1.280921,0.742706,0.742706,0.743179,0.729152,0.742706,0.848485,0.714634,0.723708,0.673511,0.707424,0.941799,0.751724,0.668852,0.606061,0.655319


Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
No improvement in 5 evaluation steps. Stopping training early.
Collected incorrect predictions for further analysis.
7/7 폴드 완료


In [ ]:
eval_result = trainer.evaluate()
fold_results.append(eval_result)

No improvement in 5 evaluation steps. Stopping training early.
Collected incorrect predictions for further analysis.


In [ ]:
# 평균 메트릭 계산 및 출력
avg_metrics = {metric: np.mean([fold[metric] for fold in fold_results]) for metric in fold_results[0].keys()}
print("Average metrics across folds:", avg_metrics)

Average metrics across folds: {'eval_loss': 1.0147147178649902, 'eval_accuracy': 0.6971706454465075, 'eval_recall_weighted': 0.6971706454465075, 'eval_f1_weighted': 0.6965093371708422, 'eval_f1_macro': 0.6829773875537155, 'eval_f1_micro': 0.6971706454465075, 'eval_f1_class_0': 0.8341836734693878, 'eval_f1_class_1': 0.645933014354067, 'eval_f1_class_2': 0.6393162393162393, 'eval_f1_class_3': 0.6479481641468683, 'eval_f1_class_4': 0.6666666666666666, 'eval_f1_class_5': 0.9358288770053476, 'eval_f1_class_6': 0.6692307692307692, 'eval_f1_class_7': 0.60431654676259, 'eval_f1_class_8': 0.5166666666666666, 'eval_f1_class_9': 0.669683257918552, 'eval_runtime': 17.1283, 'eval_samples_per_second': 132.062, 'eval_steps_per_second': 2.102, 'epoch': 5.889281507656066}


In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

def plot_training_loss(log_dir):
    event_files = [os.path.join(log_dir, f) for f in os.listdir(log_dir) if 'events' in f]
    if not event_files:
        raise ValueError("No event files found in log directory.")

    for event_file in event_files:
        losses = {'Step': [], 'Training Loss': [], 'Validation Loss': []}
        for e in tf.compat.v1.train.summary_iterator(event_file):
            for v in e.summary.value:
                if v.tag == 'train/loss':
                    losses['Training Loss'].append(v.simple_value)
                    losses['Step'].append(e.step)
                elif v.tag == 'eval/loss':
                    losses['Validation Loss'].append(v.simple_value)

        if losses['Step']:
            df = pd.DataFrame(losses).set_index('Step')
            plt.figure(figsize=(10, 5))
            plt.plot(df['Training Loss'], label='Training Loss')
            plt.plot(df['Validation Loss'], label='Validation Loss')
            plt.xlabel('Steps')
            plt.ylabel('Loss')
            plt.title('Training and Validation Loss')
            plt.legend()
            plt.grid(True)
            plt.show()

# 로그 디렉토리를 지정하여 그래프 그리기
plot_training_loss('./logs')

FileNotFoundError: [Errno 2] No such file or directory: './logs'

In [ ]:
trainer.save_model('/content/drive/MyDrive/codebert')

In [ ]:
import pandas as pd

df2 = pd.read_csv("/content/drive/MyDrive/Last_test2(feature).csv의 사본")

In [ ]:
df2 = df2[['labels', 'code']]

In [ ]:
import re

def preprocess2(df2, file_name):
    # 멀티 라인 주석 제거
    df2['code'] = df2['code'].replace(re.compile(r'/\*.*?\*/', re.DOTALL), "", regex=True)
    # 싱글 라인 주석 제거
    df2['code'] = df2['code'].replace(re.compile(r'//.*?\n'), "", regex=True)
    # angle brackets를 사용하는 include 제거
    df2['code'] = df2['code'].replace(re.compile(r'#include .*?\n'), "", regex=True)
    df2['code'] = df2['code'].replace(re.compile(r'#include "*.*?\n'), "", regex=True)
    # 매크로 정의 제거
    df2['code'] = df2['code'].replace(re.compile(r'#define .*?\n'), "", regex=True)
    # 전처리 지시문 제거
    df2['code'] = df2['code'].replace(re.compile(r'#undef\s+\w+'), "", regex=True)
    df2['code'] = df2['code'].replace(re.compile(r'#if\s+\w+'), "", regex=True)
    df2['code'] = df2['code'].replace(re.compile(r'#else\s+\w+'), "", regex=True)
    df2['code'] = df2['code'].replace(re.compile(r'#elif\s+\w+'), "", regex=True)
    df2['code'] = df2['code'].replace(re.compile(r'#endif'), "", regex=True)
    # 탭과 여러 공백을 하나의 공백으로
    df2['code'] = df2['code'].replace(re.compile(r'[\t ]+'), " ", regex=True)
    # 여러 줄바꿈을 하나로
    df2['code'] = df2['code'].replace(re.compile(r'\n\s*\n'), "", regex=True)
    df2['code'] = df2['code'].replace(re.compile(r'\n'), "", regex=True)
    df2['code'] = df2['code'].replace(re.compile(r'return*.*?;'), "", regex=True)
    df2['code'] = df2['code'].replace(re.compile(r'return;'), "", regex=True)
    # void func 로 변경
    df2['code'] = df2['code'].replace(re.compile(r'\b([a-zA-Z_]\w*)\s+([a-zA-Z_]\w*)\s*\(([^)]*)\)\s*{'), r"void func(\3){", regex=True)
    # 데이터프레임을 CSV 파일로 저장
    df2.to_csv(file_name, index=False)
    return df2  # 수정된 데이터프레임 반환


def tokenized(examples):
  return tokenizer(examples['code'], pad_to_max_length=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [ ]:
preprocess2(df2,"preprocess2.csv")
dataset = load_dataset("csv",data_files="preprocess2.csv")['train']
encoded_dataset2 = dataset.map(tokenized,remove_columns=['code'],batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/110 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
trainer.predict(encoded_dataset2, metric_key_prefix="eval")

PredictionOutput(predictions=array([[-1.4879644 ,  2.8263543 ,  5.4039693 , ...,  0.00657129,
         2.472671  ,  0.33797786],
       [-0.6595544 ,  2.2175016 ,  6.038263  , ...,  1.9420304 ,
        -0.805144  , -0.37197942],
       [-1.9109049 ,  1.7286193 ,  5.964004  , ...,  0.60648865,
         0.6577149 ,  1.3728734 ],
       ...,
       [-0.05479215, -1.3368119 , -1.80955   , ..., -0.6331065 ,
        -1.2736641 , -1.2247473 ],
       [-0.45175037, -1.3819286 , -1.8531921 , ..., -0.612824  ,
        -1.3127317 , -0.86583745],
       [-0.17418408, -1.3466207 , -1.8524216 , ..., -0.62538844,
        -1.318243  , -1.1429652 ]], dtype=float32), label_ids=array([ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  3,  3,  3,  3,  